In [ ]:
import os
#import pandas as pd
#import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import wikipediaapi

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance

from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread


app = Flask(__name__)
CORS(app)

In [ ]:
# import flask

In [ ]:
custom_headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'accept-language': 'en-GB,en;q=0.9',
}

In [ ]:
def get_request(url):
    res = requests.get(url, headers=custom_headers)
    return BeautifulSoup(res.text)

Wikipedia Amazon Warehouse Scraping

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')
page_py = wiki_wiki.page('List_of_Amazon_locations#North_America')

In [ ]:
section = page_py.section_by_title('Fulfillment and warehousing')
us_housing = section.section_by_title('North America').section_by_title('United States')
full_housing_text = us_housing.text.split('\n')[3:]
# full_housing_text

In [ ]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", 
               "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", 
               "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
               "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", 
               "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", 
               "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

amazon_warehouses = {}
sect = full_housing_text[0]
for loc in full_housing_text:
    # loc = loc.split()
    # print
    if loc in state_names:
        # print(loc)
        sect = loc
        amazon_warehouses[sect] = []
    else: 
        amazon_warehouses[sect] += [loc]

# amazon_warehouses

In [ ]:
amazon_warehouses['Michigan']

Finding Distances

In [ ]:
def get_lat_lng_nominatim(city_name):
    url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json"

    
    headers = {
        "User-Agent": "AmazonEco/1.0 (nmokh05@gmail.com)"
    }  # Replace with your actual app name and contact email
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 0:
                lat = data[0]['lat']
                lon = data[0]['lon']
                return float(lat), float(lon)
            else:
                print(f"No results found for city: {city_name}")
                return None, None
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None, None
    except Exception as e:
        print("Error fetching geolocation: ", str(e))
        return None, None